# Avaliação automatizada de LLMs utilizando DeepEval - Sumarização

In [1]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric

from IPython.display import display, Markdown

C:\Users\leona\Documents\GitHub\ollama_tests\.venv\Lib\site-packages\deepeval\__init__.py:49: UserWarning: You are using deepeval version 1.4.9, however version 1.5.0 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

## 1. Sumarização

Resumos factualmente corretos e que incluem os detalhes necessários do texto original.

**Alice’s Adventures in Wonderland** Chapter 1

Ref do livro: https://www.gutenberg.org/files/11/11-0.txt

In [3]:
livro_cp1 = ""
with open("./short_story.txt", "r", encoding="utf-8") as arquivo:
    livro_cp1 = arquivo.read()


display(Markdown(f'{livro_cp1[:492]}...\n\n...\n\n...{livro_cp1[-500:]}'))
print(len(livro_cp1))

CHAPTER I.
Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy...

...

...ate a little bit, and said anxiously to herself, “Which way? Which
way?”, holding her hand on the top of her head to feel which way it was
growing, and she was quite surprised to find that she remained the same
size: to be sure, this generally happens when one eats cake, but Alice
had got so much into the way of expecting nothing but out-of-the-way
things to happen, that it seemed quite dull and stupid for life to go
on in the common way.

So she set to work, and very soon finished off the cake.

11285


In [4]:
PROMPT = f"""
1. Read the following text and create a summary that includes the main points:

2. Who are the characters?
   - Classify them into main characters and other characters.
3. Where does the story take place?
   - Describe the main setting of the narrative.
4. What is the plot?
   - Provide an overview of the main events that define the story.

Here is the text:

{{TEXT}}

The response must strictly follow the proposed format:

# Summary
## 1. Characters:
- 
- 
- 
## 2. Story Setting:
- 
## 4. Plot:
- 

"""

prompt = PROMPT.format(TEXT = livro_cp1)
display(Markdown(f'{prompt[:1000]}...'))


1. Read the following text and create a summary that includes the main points:

2. Who are the characters?
   - Classify them into main characters and other characters.
3. Where does the story take place?
   - Describe the main setting of the narrative.
4. What is the plot?
   - Provide an overview of the main events that define the story.

Here is the text:

CHAPTER I.
Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was...

In [5]:
chat = ChatOpenAI(model="gpt-4o-mini")
response = chat.invoke(prompt)

In [6]:
display(Markdown(f'{response.content}'))

# Summary
In the first chapter of "Alice's Adventures in Wonderland," Alice grows bored while sitting with her sister and notices a White Rabbit with pink eyes, who appears to be in a hurry and possesses a watch. Driven by curiosity, Alice follows the Rabbit down a rabbit hole, where she falls for a long time, pondering various whimsical thoughts. Upon landing safely, she explores a hall filled with locked doors and discovers a tiny key and a small door that leads to a beautiful garden. However, she cannot fit through the door. After finding a bottle labeled "DRINK ME," Alice drinks from it and shrinks to a size small enough to enter the garden but realizes she left the key on the table. In her attempts to reach it, she eats a cake labeled "EAT ME," hoping it will change her size.

## 1. Characters:
- **Main Characters:**
  - Alice
  - The White Rabbit
- **Other Characters:**
  - Alice's sister
  - Dinah (Alice's cat)

## 2. Story Setting:
- The story primarily takes place in a fantastical underground world beginning with a rabbit hole and a long, low hall filled with locked doors. The hall is illuminated by lamps, and there are various objects like a three-legged glass table and shelves along the walls, leading to a small door that opens into a beautiful garden.

## 4. Plot:
- Alice, feeling bored, follows a White Rabbit and falls down a rabbit hole into a strange and whimsical place. After a long fall, she lands in a hall filled with locked doors. She discovers a tiny golden key and a little door that leads to a lovely garden but cannot fit through. Alice drinks from a bottle, shrinking herself to the right size to enter but then forgets the key on the table. In her frustration, she eats a cake hoping to grow larger or smaller to reach the key, setting her on a path of curious adventures.

### Avaliação da sumarização

In [7]:
test_case = LLMTestCase(
    input=livro_cp1, 
    actual_output=response.content)
metric = SummarizationMetric(
    model="gpt-4o-mini",
    assessment_questions = [
        "Does the summary correctly identify the main characters?",
        "Does the summary mention the setting where the story takes place?",
        "Does the summary provide an accurate overview of the plot?",
        "Does the summary highlight important relationships between the characters?",
        "Does the summary address the central themes of the story?"
    ],    
    include_reason=True)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0.7272727272727273
The score is 0.73 because the summary contains contradictions regarding the details of Alice's surroundings and her actions, diverging from the original text. Additionally, it introduces extra information about the hall's illumination and shelf contents that was not present in the source material, indicating a lack of fidelity to the original narrative.
